## Install the required modules

In [661]:
!pip3 install -r requirements.txt -q

## Import the data

In [662]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

directory_path = '/Users/brendanashton/dev/go/src/github.com/deasa/pinkbike_crawler/runs'

# Initialize an empty list to store DataFrames
dfs = []

for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
dataset = pd.concat(dfs, ignore_index=True)

dataset.head()
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17446 entries, 0 to 17445
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              17446 non-null  object 
 1   Year               16908 non-null  float64
 2   Manufacturer       17446 non-null  object 
 3   Model              17446 non-null  object 
 4   USD Price          17446 non-null  int64  
 5   Original Currency  17442 non-null  object 
 6   Condition          17434 non-null  object 
 7   Frame Size         17349 non-null  object 
 8   Wheel Size         17281 non-null  object 
 9   Front Travel       17053 non-null  object 
 10  Rear Travel        16938 non-null  object 
 11  Material           17339 non-null  object 
 12  Reason for Review  5442 non-null   object 
 13  URL                17446 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 1.9+ MB


## Data preprocessing

### Drop unnecessary columns

In [663]:
dataset = dataset.drop(columns=['Title', 'Frame Size', 'Reason for Review', 'URL', 'Original Currency'])
dataset.head()
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17446 entries, 0 to 17445
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Year          16908 non-null  float64
 1   Manufacturer  17446 non-null  object 
 2   Model         17446 non-null  object 
 3   USD Price     17446 non-null  int64  
 4   Condition     17434 non-null  object 
 5   Wheel Size    17281 non-null  object 
 6   Front Travel  17053 non-null  object 
 7   Rear Travel   16938 non-null  object 
 8   Material      17339 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 1.2+ MB


#### Clean the year - some are missing and some are strangely high

In [664]:
current_year = pd.Timestamp.now().year
min_year = 2000
max_year = pd.Timestamp.today().year 

# Calculate the mean (average) year within the bounds
mean_year = dataset[(dataset['Year'] >= min_year) & (dataset['Year'] <= max_year)]['Year'].mean()

# Replace out-of-bounds low years with NaN
dataset.loc[(dataset['Year'] < min_year), 'Year'] = np.nan

# Replace out-of-bounds years with the mean (Corrected line)
dataset.loc[(dataset['Year'] > max_year), 'Year'] = mean_year

dataset.head()

,Year,Manufacturer,Model,USD Price,Condition,Wheel Size,Front Travel,Rear Travel,Material
0,2021.0,NoManufacturer,NoModelFound,3500,Excellent - Lightly Ridden,29,160 mm,0 mm (Hardtail),Titanium
1,2022.0,Canyon,Torque,1340,Excellent - Lightly Ridden,29,NaN,NaN,Carbon Fiber
2,2023.0,Yeti,SB160,3700,Excellent - Lightly Ridden,29,NaN,NaN,Carbon Fiber
3,2023.0,Specialized,Turbo Levo Electric,2999,Excellent - Lightly Ridden,29,160 mm,150 mm,Aluminium
4,2018.0,NoManufacturer,NoModelFound,2208,"Good - Used, Mechanically Sound",29,170 mm,155 mm,Carbon Fiber


#### Extract numbers from front and rear travel

In [665]:
import re

# Extract numerical values using regular expressions
dataset['Rear Travel'] = dataset['Rear Travel'].astype(str).str.extract('(\d+)', expand=False)

# Convert to numeric, setting failed conversions to NaN
dataset['Rear Travel'] = pd.to_numeric(dataset['Rear Travel'], errors='coerce')

dataset['Rear Travel'].head()

0      0.0
1      NaN
2      NaN
3    150.0
4    155.0
Name: Rear Travel, dtype: float64

In [666]:
dataset['Front Travel'] = dataset['Front Travel'].astype(str).str.extract('(\d+)', expand=False)
dataset['Front Travel'] = pd.to_numeric(dataset['Front Travel'], errors='coerce')

dataset['Front Travel'].head()

0    160.0
1      NaN
2      NaN
3    160.0
4    170.0
Name: Front Travel, dtype: float64

In [667]:
dataset.head()

,Year,Manufacturer,Model,USD Price,Condition,Wheel Size,Front Travel,Rear Travel,Material
0,2021.0,NoManufacturer,NoModelFound,3500,Excellent - Lightly Ridden,29,160.0,0.0,Titanium
1,2022.0,Canyon,Torque,1340,Excellent - Lightly Ridden,29,NaN,NaN,Carbon Fiber
2,2023.0,Yeti,SB160,3700,Excellent - Lightly Ridden,29,NaN,NaN,Carbon Fiber
3,2023.0,Specialized,Turbo Levo Electric,2999,Excellent - Lightly Ridden,29,160.0,150.0,Aluminium
4,2018.0,NoManufacturer,NoModelFound,2208,"Good - Used, Mechanically Sound",29,170.0,155.0,Carbon Fiber


#### Replace NoManufacturer and NoModelFound with NaN

In [668]:
dataset.replace('NoModelFound', np.nan, inplace=True)
dataset.replace('NoManufacturer', np.nan, inplace=True)

dataset.head()

,Year,Manufacturer,Model,USD Price,Condition,Wheel Size,Front Travel,Rear Travel,Material
0,2021.0,NaN,NaN,3500,Excellent - Lightly Ridden,29,160.0,0.0,Titanium
1,2022.0,Canyon,Torque,1340,Excellent - Lightly Ridden,29,NaN,NaN,Carbon Fiber
2,2023.0,Yeti,SB160,3700,Excellent - Lightly Ridden,29,NaN,NaN,Carbon Fiber
3,2023.0,Specialized,Turbo Levo Electric,2999,Excellent - Lightly Ridden,29,160.0,150.0,Aluminium
4,2018.0,NaN,NaN,2208,"Good - Used, Mechanically Sound",29,170.0,155.0,Carbon Fiber


#### Remove any electric hits

In [669]:
dataset['Model'] = dataset['Model'].astype(str).str.replace(r'.*electric.*', "NaN", case=False, regex=True)
dataset.replace('nan', np.nan, inplace=True)
dataset.replace('NaN', np.nan, inplace=True)

dataset.head()

,Year,Manufacturer,Model,USD Price,Condition,Wheel Size,Front Travel,Rear Travel,Material
0,2021.0,NaN,NaN,3500,Excellent - Lightly Ridden,29,160.0,0.0,Titanium
1,2022.0,Canyon,Torque,1340,Excellent - Lightly Ridden,29,NaN,NaN,Carbon Fiber
2,2023.0,Yeti,SB160,3700,Excellent - Lightly Ridden,29,NaN,NaN,Carbon Fiber
3,2023.0,Specialized,NaN,2999,Excellent - Lightly Ridden,29,160.0,150.0,Aluminium
4,2018.0,NaN,NaN,2208,"Good - Used, Mechanically Sound",29,170.0,155.0,Carbon Fiber


### Build up manual predictions dataset

In [670]:
# Create a new DataFrame for manual predictions
manual_predictions = pd.DataFrame({
    'Year': [2019, 2021, 2021, 2022],
    'Manufacturer': ['Specialized', 'Canyon', 'Ibis', 'Specialized'],
    'Model': ['Stumpjumper', 'Spectral', 'Ripmo AF', 'Status 140'],
    'USD Price': [2000, 2500, 2000, 2000],
    'Condition': ['Good - Used, Mechanically Sound', 'Good - Used, Mechanically Sound', 'Good - Used, Mechanically Sound', 'Good - Used, Mechanically Sound'],
    'Wheel Size': ['29', '29', '29', '29'],
    'Front Travel': [150, 160, 160, 140],
    'Rear Travel': [140, 150, 147, 140],
    'Material': ['Carbon Fiber', 'Carbon Fiber', 'Aluminum', 'Aluminum'],
})

### Drop Model - it adds too much noise to the model

In [671]:
dataset = dataset.drop(columns=['Model'])
dataset.head()

,Year,Manufacturer,USD Price,Condition,Wheel Size,Front Travel,Rear Travel,Material
0,2021.0,NaN,3500,Excellent - Lightly Ridden,29,160.0,0.0,Titanium
1,2022.0,Canyon,1340,Excellent - Lightly Ridden,29,NaN,NaN,Carbon Fiber
2,2023.0,Yeti,3700,Excellent - Lightly Ridden,29,NaN,NaN,Carbon Fiber
3,2023.0,Specialized,2999,Excellent - Lightly Ridden,29,160.0,150.0,Aluminium
4,2018.0,NaN,2208,"Good - Used, Mechanically Sound",29,170.0,155.0,Carbon Fiber


In [672]:
manual_predictions = manual_predictions.drop(columns=['Model'])
manual_predictions.head()

,Year,Manufacturer,USD Price,Condition,Wheel Size,Front Travel,Rear Travel,Material
0,2019,Specialized,2000,"Good - Used, Mechanically Sound",29,150,140,Carbon Fiber
1,2021,Canyon,2500,"Good - Used, Mechanically Sound",29,160,150,Carbon Fiber
2,2021,Ibis,2000,"Good - Used, Mechanically Sound",29,160,147,Aluminum
3,2022,Specialized,2000,"Good - Used, Mechanically Sound",29,140,140,Aluminum


### De-duplicate

In [673]:
print(f"Original DataFrame shape: {dataset.shape}")

# Drop duplicate rows, keeping the first occurrence
dataset = dataset.drop_duplicates(keep='first')

print(f"Deduplicated DataFrame shape: {dataset.shape}")

Original DataFrame shape: (17446, 8)
Deduplicated DataFrame shape: (16547, 8)


### Drop all NaN

In [674]:
print(f"Original DataFrame shape: {dataset.shape}")
dataset = dataset.dropna()

# Print the shape of the original and deduplicated DataFrames

print(f"No NA DataFrame shape: {dataset.shape}")

Original DataFrame shape: (16547, 8)
No NA DataFrame shape: (13720, 8)


In [675]:
dataset.head()

,Year,Manufacturer,USD Price,Condition,Wheel Size,Front Travel,Rear Travel,Material
3,2023.0,Specialized,2999,Excellent - Lightly Ridden,29,160.0,150.0,Aluminium
5,2019.0,Yeti,2834,Excellent - Lightly Ridden,27.5 / 650B,170.0,150.0,Carbon Fiber
11,2024.0,Specialized,7000,Excellent - Lightly Ridden,29,170.0,150.0,Carbon Fiber
13,2018.0,Yeti,2500,Excellent - Lightly Ridden,27.5 / 650B,170.0,160.0,Carbon Fiber
14,2022.0,Mondraker,8750,New - Unridden/With Tags,29,160.0,160.0,Carbon Fiber


## Feature engineering

### Compute an age column, drop the year

In [676]:
from datetime import datetime

def convert_age(X):
    if 'Year' not in X.columns:
        raise KeyError("The DataFrame does not contain a 'Year' column.")
    
    # Make a copy of the DataFrame to avoid SettingWithCopyWarning
    X = X.copy()
    
    current_year = datetime.now().year
    X.loc[:, 'Age'] = current_year - X['Year']
    return X

dataset = convert_age(dataset)
# Drop the 'Year' column
dataset = dataset.drop(columns='Year')

dataset.head()

,Manufacturer,USD Price,Condition,Wheel Size,Front Travel,Rear Travel,Material,Age
3,Specialized,2999,Excellent - Lightly Ridden,29,160.0,150.0,Aluminium,1.0
5,Yeti,2834,Excellent - Lightly Ridden,27.5 / 650B,170.0,150.0,Carbon Fiber,5.0
11,Specialized,7000,Excellent - Lightly Ridden,29,170.0,150.0,Carbon Fiber,0.0
13,Yeti,2500,Excellent - Lightly Ridden,27.5 / 650B,170.0,160.0,Carbon Fiber,6.0
14,Mondraker,8750,New - Unridden/With Tags,29,160.0,160.0,Carbon Fiber,2.0


In [677]:
manual_predictions = convert_age(manual_predictions)
manual_predictions = manual_predictions.drop(columns='Year')
manual_predictions.head()

,Manufacturer,USD Price,Condition,Wheel Size,Front Travel,Rear Travel,Material,Age
0,Specialized,2000,"Good - Used, Mechanically Sound",29,150,140,Carbon Fiber,5
1,Canyon,2500,"Good - Used, Mechanically Sound",29,160,150,Carbon Fiber,3
2,Ibis,2000,"Good - Used, Mechanically Sound",29,160,147,Aluminum,3
3,Specialized,2000,"Good - Used, Mechanically Sound",29,140,140,Aluminum,2


### Put them in categories based on their travel numbers

In [678]:
# # Create a new column 'Category' that will categorize the listings by the rear travel.
# def categorize_travel(X):
#     if 'Rear Travel' not in X.columns:
#         raise KeyError("The DataFrame does not contain a 'Rear Travel' column.")
    
#     # Make a copy of the DataFrame to avoid SettingWithCopyWarning
#     X = X.copy()
    
#     # Create a new column 'Category' based on the 'Rear Travel' column
#     X.loc[(X['Rear Travel'] < 0), 'Category'] = np.nan
#     X.loc[(X['Rear Travel'] > 210), 'Category'] = np.nan
#     X.loc[(X['Rear Travel'] == 0), 'Category'] = 'Hardtail'
#     X.loc[(X['Rear Travel'] >= 0) & (X['Rear Travel'] < 120), 'Category'] = 'Short Travel'
#     X.loc[(X['Rear Travel'] >= 120) & (X['Rear Travel'] <= 150), 'Category'] = 'Mid Travel'
#     X.loc[(X['Rear Travel'] > 150) & (X['Rear Travel'] <= 210), 'Category'] = 'Long Travel'
#     return X

# print(f"Original DataFrame shape: {dataset.shape}")
# dataset = categorize_travel(dataset)
# dataset = dataset.dropna()
# print(f"DataFrame shape after categorization: {dataset.shape}")

# dataset.head()

In [679]:
# manual_predictions = categorize_travel(manual_predictions)
# manual_predictions.head()

### Drop now-irrelevant front travel and rear travel columns

In [680]:
# dataset = dataset.drop(columns=['Rear Travel', 'Front Travel'])
# dataset.head()

In [681]:
# manual_predictions = manual_predictions.drop(columns=['Rear Travel', 'Front Travel'])
# manual_predictions.head()

### Reorder columns so the target variable is at the end

In [682]:
dataset = dataset[[col for col in dataset.columns if col != 'USD Price'] + ['USD Price']]
dataset.head()

,Manufacturer,Condition,Wheel Size,Front Travel,Rear Travel,Material,Age,USD Price
3,Specialized,Excellent - Lightly Ridden,29,160.0,150.0,Aluminium,1.0,2999
5,Yeti,Excellent - Lightly Ridden,27.5 / 650B,170.0,150.0,Carbon Fiber,5.0,2834
11,Specialized,Excellent - Lightly Ridden,29,170.0,150.0,Carbon Fiber,0.0,7000
13,Yeti,Excellent - Lightly Ridden,27.5 / 650B,170.0,160.0,Carbon Fiber,6.0,2500
14,Mondraker,New - Unridden/With Tags,29,160.0,160.0,Carbon Fiber,2.0,8750


In [683]:
manual_predictions = manual_predictions[[col for col in manual_predictions.columns if col != 'USD Price'] + ['USD Price']]
manual_predictions.head()

,Manufacturer,Condition,Wheel Size,Front Travel,Rear Travel,Material,Age,USD Price
0,Specialized,"Good - Used, Mechanically Sound",29,150,140,Carbon Fiber,5,2000
1,Canyon,"Good - Used, Mechanically Sound",29,160,150,Carbon Fiber,3,2500
2,Ibis,"Good - Used, Mechanically Sound",29,160,147,Aluminum,3,2000
3,Specialized,"Good - Used, Mechanically Sound",29,140,140,Aluminum,2,2000


### Split the data into features and a target variable

In [684]:
X = dataset.iloc[:, :-1]
Y = dataset.iloc[:, -1]
X_manual = manual_predictions.iloc[:, :-1]
Y_manual = manual_predictions.iloc[:, -1]
manual_predictions_copy = manual_predictions.copy()

In [685]:
print(X)

         Manufacturer                        Condition   Wheel Size  \
3         Specialized       Excellent - Lightly Ridden           29   
5                Yeti       Excellent - Lightly Ridden  27.5 / 650B   
11        Specialized       Excellent - Lightly Ridden           29   
13               Yeti       Excellent - Lightly Ridden  27.5 / 650B   
14          Mondraker         New - Unridden/With Tags           29   
...               ...                              ...          ...   
17441  Rocky Mountain       Excellent - Lightly Ridden  27.5 / 650B   
17442      Santa Cruz  Good - Used, Mechanically Sound           29   
17443     Diamondback       Excellent - Lightly Ridden           29   
17444            Kona  Good - Used, Mechanically Sound           29   
17445           Giant  Good - Used, Mechanically Sound           29   

       Front Travel  Rear Travel      Material  Age  
3             160.0        150.0     Aluminium  1.0  
5             170.0        150.0  Carbo

In [686]:
print(Y)

3        2999
5        2834
11       7000
13       2500
14       8750
         ... 
17441    2500
17442    3000
17443     800
17444    2850
17445    2700
Name: USD Price, Length: 13720, dtype: int64


In [687]:
print(X_manual)

  Manufacturer                        Condition Wheel Size  Front Travel  \
0  Specialized  Good - Used, Mechanically Sound         29           150   
1       Canyon  Good - Used, Mechanically Sound         29           160   
2         Ibis  Good - Used, Mechanically Sound         29           160   
3  Specialized  Good - Used, Mechanically Sound         29           140   

   Rear Travel      Material  Age  
0          140  Carbon Fiber    5  
1          150  Carbon Fiber    3  
2          147      Aluminum    3  
3          140      Aluminum    2  


### Put non-categorical columns at the beginning for easier viewing

In [688]:
X = X[['Rear Travel'] + [col for col in X.columns if col != 'Rear Travel']]
X_manual = X_manual[['Rear Travel'] + [col for col in X_manual.columns if col != 'Rear Travel']]
X = X[['Front Travel'] + [col for col in X.columns if col != 'Front Travel']]
X_manual = X_manual[['Front Travel'] + [col for col in X_manual.columns if col != 'Front Travel']]
X = X[['Age'] + [col for col in X.columns if col != 'Age']]
X_manual = X_manual[['Age'] + [col for col in X_manual.columns if col != 'Age']]
X.head()

,Age,Front Travel,Rear Travel,Manufacturer,Condition,Wheel Size,Material
3,1.0,160.0,150.0,Specialized,Excellent - Lightly Ridden,29,Aluminium
5,5.0,170.0,150.0,Yeti,Excellent - Lightly Ridden,27.5 / 650B,Carbon Fiber
11,0.0,170.0,150.0,Specialized,Excellent - Lightly Ridden,29,Carbon Fiber
13,6.0,170.0,160.0,Yeti,Excellent - Lightly Ridden,27.5 / 650B,Carbon Fiber
14,2.0,160.0,160.0,Mondraker,New - Unridden/With Tags,29,Carbon Fiber


## Transform the data with one-hot encoding

In [689]:
def encode_data(X):
    for column in ['Manufacturer', 'Condition', 'Wheel Size', 'Material']:#, 'Category']:
        insert_loc = X.columns.get_loc(column)
        X = pd.concat([X.iloc[:,:insert_loc], pd.get_dummies(X.loc[:, [column]]), X.iloc[:,insert_loc+1:]], axis=1)
    return X

# temporarily join the X and X_manual DataFrames to encode the categorical variables
X = pd.concat([X, X_manual], ignore_index=True)
X = np.array(encode_data(X.copy()))

# Split the encoded data back into the original X and X_manual DataFrames
X, X_manual = X[:len(dataset)], X[len(dataset):]

print(X_manual)

import pandas.testing as tm
actual_data = pd.DataFrame({
    'Age': pd.DataFrame(X_manual).iloc[:, 0],
    'Front Travel': pd.DataFrame(X_manual).iloc[:, 1],
    'Rear Travel': pd.DataFrame(X_manual).iloc[:, 2],
})
expected_data = pd.DataFrame({
    'Age': [5.0, 3.0, 3.0, 2.0],
    'Front Travel': [150.0, 160.0, 160.0, 140.0],
    'Rear Travel': [140.0, 150.0, 147.0, 140.0],
})

tm.assert_frame_equal(actual_data, expected_data, check_dtype=False)


[[5.0 150.0 140.0 False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False True
  False False False False False False False False False False False False
  True False False False False False False True False False False False
  True False False False]
 [3.0 160.0 150.0 False False False False False False True False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  True False False F

## Apply feature scaling
We don't actually have to apply feature scaling for decision tree regression because the independent variable raw values aren't being compared to eachother so they don't need to be scaled.

## Train the model

In [690]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state=0)
regressor.fit(X, Y)

DecisionTreeRegressor(random_state=0)

## Predicting the test set results

In [695]:
regressor.predict(X_manual)

for i, prediction in enumerate(regressor.predict(X_manual)):
    row_data = manual_predictions_copy.iloc[i]
    print(f"Prediction for bike {i + 1}: ${prediction:.2f}, Data: {row_data.to_dict()}")

Prediction for bike 1: $2634.40, Data: {'Manufacturer': 'Specialized', 'Condition': 'Good - Used, Mechanically Sound', 'Wheel Size': '29', 'Front Travel': 150, 'Rear Travel': 140, 'Material': 'Carbon Fiber', 'Age': 5, 'USD Price': 2000}
Prediction for bike 2: $3462.25, Data: {'Manufacturer': 'Canyon', 'Condition': 'Good - Used, Mechanically Sound', 'Wheel Size': '29', 'Front Travel': 160, 'Rear Travel': 150, 'Material': 'Carbon Fiber', 'Age': 3, 'USD Price': 2500}
Prediction for bike 3: $2142.50, Data: {'Manufacturer': 'Ibis', 'Condition': 'Good - Used, Mechanically Sound', 'Wheel Size': '29', 'Front Travel': 160, 'Rear Travel': 147, 'Material': 'Aluminum', 'Age': 3, 'USD Price': 2000}
Prediction for bike 4: $2229.80, Data: {'Manufacturer': 'Specialized', 'Condition': 'Good - Used, Mechanically Sound', 'Wheel Size': '29', 'Front Travel': 140, 'Rear Travel': 140, 'Material': 'Aluminum', 'Age': 2, 'USD Price': 2000}
